In [59]:
pos = [0, 0]
board = [
    'b---0',
    '----0',
    '----0',
    '----0',
    '----0',
]

In [61]:
from itertools import permutations
import os
import pickle

def findB(grid):
    for r, row in enumerate(grid):
        for c, val in enumerate(row):
            if val == 'b':
                return (r, c)
    return False

def findD(grid):
    d = []
    for r, row in enumerate(grid):
        for c, val in enumerate(row):
            if val == 'd':
                d.append((r, c))
    return d    

def findChars(grid, chars):
    d = []
    for r, row in enumerate(grid):
        for c, val in enumerate(row):
            if val in chars:
                d.append((r, c))
    return d    

def findPathLength(x, y):
    return abs(x[0] - y[0]) + abs(x[1] - y[1])

def findMinPath(b, ds):
    sums = {}
    if b in ds:
        return [b, b]
    for d in permutations(ds):
        d = [b] + list(d)
        pairs = [(d[i], d[i+1]) for i in range(len(d) - 1)]
        sums[sum([findPathLength(i[0], i[1]) for i in pairs])] = d
    return sums[min(list(sums.keys()))]

def createSteps(path):
    steps = []
    for i in range(len(path) - 1):
        a = path[i]
        b = path[i + 1]
        dx = b[1] - a[1]
        dy = b[0] - a[0]
        if dx > 0: steps = steps + ['RIGHT'] * dx
        if dx < 0: steps = steps + ['LEFT'] * (-1 * dx)
        if dy > 0: steps = steps + ['DOWN'] * dy
        if dy < 0: steps = steps + ['UP'] * (-1 * dy)   
        steps = steps + ['CLEAN']
    return steps

def modifyBoard(posr, posc, board, step):
    if step == 'RIGHT':
        board[posr] = board[posr].replace('b-', '-b').replace('bd', '-d')
        posc = posc + 1
    if step == 'LEFT':
        board[posr] = board[posr].replace('-b', 'b-').replace('db', 'd-')
        posc = posc - 1
    if step == 'DOWN':
        board_row_up = [i for i in board[posr]]
        board_row_down = [i for i in board[posr + 1]]
        if board_row_down[posc] == '-':
            board_row_down[posc] = 'b'
        board_row_up[posc] = '-'
        board[posr] = ''.join(board_row_up)
        board[posr + 1] = ''.join(board_row_down)
        posr = posr + 1 
    if step == 'UP':
        board_row_up = [i for i in board[posr - 1]]
        board_row_down = [i for i in board[posr]]
        if board_row_up[posc] == '-':
            board_row_up[posc] = 'b'
        board_row_down[posc] = '-'
        board[posr - 1] = ''.join(board_row_up)
        board[posr] = ''.join(board_row_down)
        posr = posr - 1
    if step == 'CLEAN':
        board_row = [i for i in board[posr]]
        board_row[posc] = 'b'
        board[posr] = ''.join(board_row)
        
    return posr, posc, board

def getSubBoard(b, ds, count=1):
    sub_board = ds
    for i in range(len(board) + len(board[0])):
        sub_board = [dot for dot in ds if findPathLength(b, dot) < i ]
        if len(sub_board) >= count:
            break
    return sub_board

def getUnknownCells(board):
    try:
        with open('data.pickle', 'rb') as f:
            unknowns = pickle.load(f)
    except:
        unknowns = []
        for j in range(len(board)):
            for i in range(len(board[0])):
                unknowns.append((i, j))
        with open('data.pickle', 'wb') as f:
            pickle.dump(unknowns, f)
    
    chars = findChars(board, ['b', '-'])
    unknowns = [i for i in unknowns if i not in chars]
    
    with open('data.pickle', 'wb') as f:
        pickle.dump(unknowns, f)
    
    return unknowns
     
def getArrayFromBoard(board):
    return [[c for c in r] for r in board]

def getBoardFromArray(array):
    return [''.join(r) for r in array]
    
def replaceUnknown(board, unknown):
    array = getArrayFromBoard(board)
    for r, row in enumerate(array):
        for c, cell in enumerate(row):
            if (r, c) in unknown:
                array[r][c] = 'd'
    return getBoardFromArray(array)
    
def next_move(posr, posc, board):
    unknown = getUnknownCells(board)
    b = findB(board) or (posr, posc)

    ds = getSubBoard(b, findChars(board, ['d']), 1)
    if not len(ds):
        board = replaceUnknown(board, unknown)
        ds = getSubBoard(b, findChars(board, ['d']), 5)
    
#     print(unknown)
#     print(ds)
#     print(board)
    
    path = findMinPath(b, ds)
    steps = createSteps(path)
    print(steps[0])

if __name__ == "__main__": 
    pos = [int(i) for i in input().strip().split()] 
    board = [[j for j in input().strip()] for i in range(5)]  
    next_move(pos[0], pos[1], board)
    

[(0, 4), (1, 4), (2, 4), (3, 4), (4, 4)]
[(0, 4)]
['b---d', '----d', '----d', '----d', '----d']
RIGHT
